# Iteration 4: includes 4 types of patients , 4 types of LOS distributions and tracks the number of patients in each state at each time step.

---

## **. Patient Admission Sources**  
Patients enter the hospital through **two primary pathways**:  
- **New Admissions**: Direct hospital entry for Stroke, TIA, Complex Neurological, and Other Medical Cases.  


| Patient Type                     | Admissions (n) | Percentage (%) |
|----------------------------------|---------------|--------------|
| Stroke                           | 1,320         | 54%          |
| Transient Ischemic Attack (TIA)  | 158           | 6%           |
| Complex Neurological             | 456           | 19%          |
| Other Medical Cases              | 510           | 21%          |

**Patient admissions are distributed as follows**

| Category                | Mean(Days) |
|-------------------------|----------------------------|
| Stroke                  | 1.2                        |
| TIA (Transient Ischemic Attack) | 9.3                |
| Complex Neurological    | 3.6                        |
| Other                   | 3.2                        |



## 1. Imports 

In [5]:
import numpy as np
import itertools
import simpy
import os
import sys

In [6]:
# Add parent directory to path so we can import distribution.py
sys.path.append(os.path.abspath('..'))  # noqa: E402

from distribution import Exponential, Lognormal  # noqa: E402

## 2. Constants

In [7]:
# default mean inter-arrival times(exp)
IAT_STROKE = 1.2
IAT_TIA = 9.3
IAT_COMPLEX_NEURO = 3.6
IAT_OTHER = 3.2

# Default Length of Stay (LOS) parameters
# (mean, stdev for Lognormal distribution
LOS_STROKE = (7.4, 8.6)
# LOS_STROKE_NESD = (7.4, 8.6)
# LOS_STROKE_ESD = (4.6, 4.8)
# LOS_STROKE_MORTALITY = (7.0, 8.7)
LOS_TIA = (1.8, 2.3)
LOS_COMPLEX_NEURO = (4.0, 5.0)
LOS_OTHER = (3.8, 5.2)

# sampling settings, 4 for arrivals, 4 for LOS
N_STREAMS = 8
DEFAULT_RND_SET = 0

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables (units = hours)
RUN_LENGTH = 24 * 10

## 2. Helper classes and functions

In [8]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Experiment class

In [9]:
class Experiment:
    """
    Encapsulates the concept of an experiment for
    the Acute Stroke Unit simulation.
    Manages parameters, PRNG streams, and results.
    """

    def __init__(
        self,
        random_number_set=0,
        n_streams=N_STREAMS,
        iat_stroke=IAT_STROKE,
        iat_tia=IAT_TIA,
        iat_complex_neuro=IAT_COMPLEX_NEURO,
        iat_other=IAT_OTHER,
        asu_beds=10,
        los_stroke=LOS_STROKE,
        los_tia=LOS_TIA,
        los_complex_neuro=LOS_COMPLEX_NEURO,
        los_other=LOS_OTHER,
    ):
        """
        Initialize default parameters.
        """
        # Sampling settings
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # Model parameters
        self.iat_stroke = iat_stroke
        self.iat_tia = iat_tia
        self.iat_complex_neuro = iat_complex_neuro
        self.iat_other = iat_other
        self.asu_beds = asu_beds

        # LOS Parameters
        self.los_stroke = los_stroke
        self.los_tia = los_tia
        self.los_complex_neuro = los_complex_neuro
        self.los_other = los_other

        # Initialize results storage
        self.init_results_variables()

        # Initialize sampling distributions
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling.

        Parameters:
        ----------
        random_number_set: int
            Controls the set of pseudo-random
            numbers used by the distributions.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Creates the distributions used by the model and initializes
        the random seeds of each.
        """
        # Produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # Inter-arrival time distributions
        self.arrival_stroke = Exponential(
            self.iat_stroke, random_seed=self.seeds[0]
        )
        self.arrival_tia = Exponential(self.iat_tia, random_seed=self.seeds[1])
        self.arrival_complex_neuro = Exponential(
            self.iat_complex_neuro, random_seed=self.seeds[2]
        )
        self.arrival_other = Exponential(
            self.iat_other, random_seed=self.seeds[3]
        )

        # LOS distributions using stored parameters
        self.los_distributions = {
            "stroke": Lognormal(*self.los_stroke, random_seed=self.seeds[4]),
            "tia": Lognormal(*self.los_tia, random_seed=self.seeds[5]),
            "complex_neuro": Lognormal(
                *self.los_complex_neuro, random_seed=self.seeds[6]
            ),
            "other": Lognormal(*self.los_other, random_seed=self.seeds[7]),
        }

    def init_results_variables(self):
        """
        Initializes all the experiment variables used in results collection.
        """
        self.results = {}
        self.results["n_stroke"] = 0
        self.results["n_tia"] = 0
        self.results["n_complex_neuro"] = 0
        self.results["n_other"] = 0
        self.results["n_patients"] = 0
        self.results["n_stroke_discharged"] = 0
        self.results["n_tia_discharged"] = 0
        self.results["n_complex_neuro_discharged"] = 0
        self.results["n_other_discharged"] = 0
        self.results["n_discharged"] = 0

## 3. Patient Class

In [10]:
class Patient:
    """
    Represents a patient in the system.
    """

    def __init__(self, patient_id, env, args, acute_stroke_unit, patient_type):
        self.patient_id = patient_id
        self.env = env
        self.args = args
        self.acute_stroke_unit = acute_stroke_unit  # Pass the ASU instance
        self.patient_type = patient_type

    def treatment(self):
        """
        Simulates patient treatment process.
        """
        arrival_time = self.env.now
        los_distribution = self.args.los_distributions[self.patient_type]

        # Arrival message
        trace(
            f"Patient {self.patient_id} ({self.patient_type.upper()}) \
            arrives at {self.env.now:.2f}."
        )

        # Request bed
        with self.acute_stroke_unit.beds.request() as request:
            yield request
            waiting_time = self.env.now - arrival_time
            los = los_distribution.sample()  # Sample LOS from distribution
            # tested los matched
            # print(los)

            # Bed assigned message
            trace(
                f"Patient {self.patient_id} ({self.patient_type.upper()}) "
                f" gets a bed at {self.env.now:.2f}."
                f" waiting time: {waiting_time:.2f} days."
            )

            # Simulate length of stay
            yield self.env.timeout(los)  # Patient stays in bed

            # Track patient discharge
            self.args.results[f"n_{self.patient_type}_discharged"] += 1
            self.args.results["n_discharged"] += 1

            # Leaving message
            trace(
                f"Patient {self.patient_id} ({self.patient_type.upper()}) \
                leaves at {self.env.now:.2f}."
            )
            trace(
                f"Patient {self.patient_id} ({self.patient_type.upper()}) \
                has a length of stay of {los:.2f} days."
            )

### 4. Acute Stroke Unit Class


In [11]:
class AcuteStrokeUnit:
    """
    Models the Acute Stroke Unit (ASU) in the hospital.
    """

    def __init__(self, env, args):
        self.env = env
        self.args = args
        self.beds = simpy.Resource(env, capacity=args.asu_beds)

    def patient_arrivals(self, patient_type, arrival_distribution):
        """
        Generic arrival generator for patients.
        """
        for patient_count in itertools.count(start=1):
            inter_arrival_time = arrival_distribution.sample()
            yield self.env.timeout(inter_arrival_time)

            # Track patient count for each type
            self.args.results[f"n_{patient_type}"] += 1
            self.args.results["n_patients"] += 1
            trace(f"{self.env.now:.2f}: {patient_type.upper()} arrival.")

            # Create new patient and process their treatment
            new_patient = Patient(
                patient_count, self.env, self.args, self, patient_type
            )
            self.env.process(new_patient.treatment())

## 4. A function per arrival source

The first approach we will use is creating an arrival generator per source.  There will be some code redundancy, but it will a clear design for others to understand.

## 5. Single run function

In [12]:
def single_run(experiment, rep=0, run_length=RUN_LENGTH):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/parameters to use with model

    rep: int
        The replication number.

    run_length: float, optional (default=RUN_LENGTH)
        The run length of the model
    """

    # Reset results for each run
    experiment.init_results_variables()

    # Set the random number set for the run
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()
    asu = AcuteStrokeUnit(env, experiment)  # Create ASU instance

    # Create patient arrival processes for different types of patients
    env.process(asu.patient_arrivals("stroke", experiment.arrival_stroke))
    env.process(asu.patient_arrivals("tia", experiment.arrival_tia))
    env.process(
        asu.patient_arrivals("complex_neuro", experiment.arrival_complex_neuro)
    )
    env.process(asu.patient_arrivals("other", experiment.arrival_other))

    # run for warm-up + results collection period
    env.run(until=run_length)

    # Final summary after the simulation ends
    total_patients = sum(
        experiment.results[key]
        for key in experiment.results
        if key.startswith("n_")
    )
    trace(
        f"Final summary: There were a total of {total_patients} \
        patients in the system."
    )

    # return the count of the arrivals
    return experiment.results

In [13]:
TRACE = True
experiment = Experiment()
results = single_run(experiment)
results

3.95: STROKE arrival.
Patient 1 (STROKE)             arrives at 3.95.
Patient 1 (STROKE)  gets a bed at 3.95. waiting time: 0.00 days.
4.64: OTHER arrival.
Patient 1 (OTHER)             arrives at 4.64.
Patient 1 (OTHER)  gets a bed at 4.64. waiting time: 0.00 days.
4.87: STROKE arrival.
Patient 2 (STROKE)             arrives at 4.87.
Patient 2 (STROKE)  gets a bed at 4.87. waiting time: 0.00 days.
4.95: OTHER arrival.
Patient 2 (OTHER)             arrives at 4.95.
Patient 2 (OTHER)  gets a bed at 4.95. waiting time: 0.00 days.
5.51: COMPLEX_NEURO arrival.
Patient 1 (COMPLEX_NEURO)             arrives at 5.51.
Patient 1 (COMPLEX_NEURO)  gets a bed at 5.51. waiting time: 0.00 days.
5.92: COMPLEX_NEURO arrival.
Patient 2 (COMPLEX_NEURO)             arrives at 5.92.
Patient 2 (COMPLEX_NEURO)  gets a bed at 5.92. waiting time: 0.00 days.
Patient 1 (OTHER)                 leaves at 6.14.
Patient 1 (OTHER)                 has a length of stay of 1.50 days.
Patient 2 (STROKE)                 

{'n_stroke': 200,
 'n_tia': 19,
 'n_complex_neuro': 60,
 'n_other': 80,
 'n_patients': 359,
 'n_stroke_discharged': 189,
 'n_tia_discharged': 19,
 'n_complex_neuro_discharged': 58,
 'n_other_discharged': 75,
 'n_discharged': 341}

## TEST - Should only show TIA patients

In [14]:
M = 1_000_000
experiment = Experiment(iat_stroke=M, iat_complex_neuro=M, iat_other=M)
results = single_run(experiment)
results

7.47: TIA arrival.
Patient 1 (TIA)             arrives at 7.47.
Patient 1 (TIA)  gets a bed at 7.47. waiting time: 0.00 days.
Patient 1 (TIA)                 leaves at 8.52.
Patient 1 (TIA)                 has a length of stay of 1.06 days.
11.02: TIA arrival.
Patient 2 (TIA)             arrives at 11.02.
Patient 2 (TIA)  gets a bed at 11.02. waiting time: 0.00 days.
Patient 2 (TIA)                 leaves at 11.33.
Patient 2 (TIA)                 has a length of stay of 0.31 days.
14.41: TIA arrival.
Patient 3 (TIA)             arrives at 14.41.
Patient 3 (TIA)  gets a bed at 14.41. waiting time: 0.00 days.
Patient 3 (TIA)                 leaves at 14.78.
Patient 3 (TIA)                 has a length of stay of 0.37 days.
19.33: TIA arrival.
Patient 4 (TIA)             arrives at 19.33.
Patient 4 (TIA)  gets a bed at 19.33. waiting time: 0.00 days.
Patient 4 (TIA)                 leaves at 20.48.
Patient 4 (TIA)                 has a length of stay of 1.15 days.
45.26: TIA arrival.
Pati

{'n_stroke': 0,
 'n_tia': 19,
 'n_complex_neuro': 0,
 'n_other': 0,
 'n_patients': 19,
 'n_stroke_discharged': 0,
 'n_tia_discharged': 19,
 'n_complex_neuro_discharged': 0,
 'n_other_discharged': 0,
 'n_discharged': 19}